In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine
from scipy.stats import bartlett
from scipy.stats import levene
import statsmodels.api as sm


# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.2f}'.format

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()

weather.head(10)

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.47,7.39,0.89,14.12,251.00,15.83,0.00,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.36,7.23,0.86,14.26,259.00,15.83,0.00,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.38,9.38,0.89,3.93,204.00,14.96,0.00,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.29,5.94,0.83,14.10,269.00,15.83,0.00,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.76,6.98,0.83,11.04,259.00,15.83,0.00,1016.51,Partly cloudy throughout the day.
5,2006-04-01 03:00:00+00:00,Partly Cloudy,rain,9.22,7.11,0.85,13.96,258.00,14.96,0.00,1016.66,Partly cloudy throughout the day.
6,2006-04-01 04:00:00+00:00,Partly Cloudy,rain,7.73,5.52,0.95,12.36,259.00,9.98,0.00,1016.72,Partly cloudy throughout the day.
7,2006-04-01 05:00:00+00:00,Partly Cloudy,rain,8.77,6.53,0.89,14.15,260.00,9.98,0.00,1016.84,Partly cloudy throughout the day.
8,2006-04-01 06:00:00+00:00,Partly Cloudy,rain,10.82,10.82,0.82,11.32,259.00,9.98,0.00,1017.37,Partly cloudy throughout the day.
9,2006-04-01 07:00:00+00:00,Partly Cloudy,rain,13.77,13.77,0.72,12.53,279.00,9.98,0.00,1017.22,Partly cloudy throughout the day.


In [2]:
#Build OLS model
Y=weather.temperature - weather.apparenttemperature

X=weather[['humidity', 'windspeed']]

X=sm.add_constant(X)

results= sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 19 Sep 2019   Prob (F-statistic):               0.00
Time:                        16:51:51   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.4381      0.021   -115.948      0.000      -2.479      -2.397
humidity       3.0292      0.024    126.479      0.000       2.982       3.076
windspeed      0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Both coefficients are statistically significant. It would stand to reason that windspeed would have a negative coefficient, as apparent temperatures usually decrease as windspeed increases, while humidity reaches its maximum during rain, at which point temperature decreases. Since the coefficient for humidity (3.03) is much higher than windspeed(.12), a point change in humidity will affect the target variable much more than a change in humidity. 

In [3]:
weather['humiditywind']= weather.humidity * weather.windspeed
X=weather[['humidity', 'windspeed', 'humiditywind']]

X=sm.add_constant(X)

results= sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 19 Sep 2019   Prob (F-statistic):               0.00
Time:                        16:51:51   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0839      0.033     -2.511      0.012      -0.149      -0.018
humidity        -0.1775      0.043     -4.133      0.000      -0.262      -0.093
windspeed       -0.0905      0.002    -36.797      0.000      -0.095      -0.086
humiditywind     0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Weather Analysis
With the humidity/windspeed interactive feature added, all coefficients are still significant, but humidity and windspeed's estimated coefficients have become negative. Since the interaction term is statistically significant and the coefficient of determination(R squared) has increased, the interaction term has predictive power in the model and must be kept. 0.2971, the interaction term coefficient, demonstrates that the greater value the humidity holds, the greater effect that windspeed will have on the target, and vice versa. - .1775 represents the effect humidity would have on the target if windspeed were held at 0, and -.0905 represents the change in the target variable for each one unit change in windspeed if humidity were held at 0. 

In [4]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houses= pd.read_sql_query('select * from houseprices', con=engine)
# no need for an open connection, as we're only doing a single query
engine.dispose()

houses.head(10)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.00,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.00,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.00,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.00,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.00,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000
5,6,50,RL,85.00,14115,Pave,None,IR1,Lvl,AllPub,...,0,None,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.00,10084,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,8,2007,WD,Normal,307000
7,8,60,RL,nan,10382,Pave,None,IR1,Lvl,AllPub,...,0,None,None,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.00,6120,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.00,7420,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,1,2008,WD,Normal,118000


In [5]:
#Cleaned variables
from scipy.stats.mstats import winsorize
houses.garagecars= winsorize(houses.garagecars, (0, 0.05))
houses.grlivarea= winsorize(houses.grlivarea, (0, 0.05))
houses.totalbsmtsf= winsorize(houses.totalbsmtsf, (0, 0.05))
houses.firstflrsf= winsorize(houses.firstflrsf, (0, 0.05))

In [6]:
Y=houses.saleprice

X=houses[['overallqual', 'garagecars', 'grlivarea', 'totalbsmtsf', 'firstflrsf']]

X=sm.add_constant(X)

results= sm.OLS(Y,X).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:              saleprice   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                     923.0
Date:                Thu, 19 Sep 2019   Prob (F-statistic):               0.00
Time:                        16:51:52   Log-Likelihood:                -17501.
No. Observations:                1460   AIC:                         3.501e+04
Df Residuals:                    1454   BIC:                         3.505e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.188e+05   5003.870    -23.748      

# Houses Analysis
All features are statistically significant at the .001 level. Overallqual's coefficient is the largest, meaning it has the strongest effect on saleprice, followed, by garagecars, then grlivarea, totalbsmtsf, and finally firstflrsf. The results seem somewhat reasonable, as overallqual must be an aggregate variable already closely related to how a realtor perceives value in a home, and garagecars may be a strong predictor for home size: very few homes would have a three car garage and be cramped for space and quality. It seems wrong at first glance that basement square footage would have nearly twice the effect on saleprice than first floor square footage. 